# 6. Generazione delle Visualizzazioni per la Presentazione

**Obiettivo:** Creare e salvare una serie di grafici di alta qualità, pensati per comunicare i risultati chiave del mio progetto in una presentazione. Questo è l'ultimo passo, focalizzato sulla comunicazione visiva.

**Logica:** Ogni cella genera una visualizzazione specifica per una slide, e la salva come immagine PNG.

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pyspark.sql.functions import avg, col, desc

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.clustering.models import get_cluster_profiles
from src.config.spark_config import SPARK_CONFIG
from src.utils.helpers import get_spark_session

project_root = os.path.abspath(os.path.join('..'))
processed_data_dir = os.path.join(project_root, "data", "processed")
adv_metrics_path = os.path.join(processed_data_dir, "players_advanced_metrics.parquet")
clustered_path = os.path.join(processed_data_dir, "players_clustered.parquet")
FIG_OUTPUT_PATH = os.path.join(project_root, "reports", "figures")
os.makedirs(FIG_OUTPUT_PATH, exist_ok=True)

spark = get_spark_session(
    app_name="NBA_Presentation_Visuals",
    driver_memory=SPARK_CONFIG["driver_memory"]
)

sns.set_theme(style="whitegrid")
print(f"I grafici verranno salvati in: {os.path.abspath(FIG_OUTPUT_PATH)}")

### Caricamento dei Dati Necessari

Carico sia il dataset con le metriche avanzate sia quello con i risultati del clustering.

In [ ]:
df_adv_metrics = spark.read.parquet(adv_metrics_path)
df_clustered = spark.read.parquet(clustered_path)
df_clustered = df_clustered.withColumn("cluster_id", col("cluster_id") + 1)

### Grafico 1: L'Effetto della Normalizzazione

**Scopo:** Mostrare perché la normalizzazione "per 36 minuti" è fondamentale per un confronto equo.

In [ ]:
df_2023 = df_adv_metrics.filter(col('season') == 2023)

starter = df_2023.orderBy(desc("mp")).first()
median_mp = df_2023.approxQuantile("mp", [0.5], 0.01)[0]
reserve = df_2023.filter(col('mp') >= median_mp).orderBy("mp").first()

plot_data = pd.DataFrame([
    {'player': starter['player'], 'type': 'Punti Totali', 'value': starter['pts']},
    {'player': starter['player'], 'type': 'Punti per 36 min', 'value': starter['pts_per_36_min']},
    {'player': reserve['player'], 'type': 'Punti Totali', 'value': reserve['pts']},
    {'player': reserve['player'], 'type': 'Punti per 36 min', 'value': reserve['pts_per_36_min']}
])

plt.figure(figsize=(10, 6))
sns.barplot(data=plot_data, x='player', y='value', hue='type', palette=['#ff7f0e', '#1f77b4'])
plt.title('Effetto della Normalizzazione: Titolare vs. Riserva', fontsize=16)
plt.ylabel('Valore Statistico (Punti)')
plt.xlabel('Giocatore')
plt.legend(title='Tipo di Statistica')
plt.tight_layout()
plt.savefig(os.path.join(FIG_OUTPUT_PATH, 'slide_7_normalization_effect.png'))
plt.show()

#### Interpretazione del grafico

Il grafico mostra bene come il giocatore di riserva, pur avendo meno punti totali, mostri un impatto per 36 minuti comparabile a quello del titolare. Questo dimostra l'importanza della normalizzazione.

### Grafico 2: Top 10 Giocatori per True Shooting % (TS%)

**Scopo:** Evidenziare i giocatori più efficienti al tiro.

In [ ]:
top_ts_players = df_adv_metrics.filter((col('season') == 2023) & (col('fga') > 250)) \
                                .orderBy(desc("ts_pct_calc")) \
                                .limit(10) \
                                .toPandas()

plt.figure(figsize=(12, 8))
sns.barplot(data=top_ts_players, x='ts_pct_calc', y='player', palette='viridis')
plt.title('Top 10 Giocatori per Efficienza al Tiro (TS%) - Stagione 2023', fontsize=16)
plt.xlabel('True Shooting Percentage (TS%)')
plt.ylabel('Giocatore')
plt.tight_layout()
plt.savefig(os.path.join(FIG_OUTPUT_PATH, 'slide_8_top_ts_players.png'))
plt.show()

#### Interpretazione del grafico

Questo grafico mostra chiaramente chi sono i finalizzatori più efficienti della lega, una metrica chiave che ho usato anche nel mio clustering.

### Grafici 3-8: Confronto per Ciascun Cluster

**Scopo:** Creare un grafico per ogni cluster che ne evidenzi le peculiarità rispetto alla media.

In [ ]:
feature_cols = [
    'pts_per_36_min', 'trb_per_36_min', 'ast_per_36_min', 
    'stl_per_36_min', 'blk_per_36_min', 'tov_per_36_min',
    'ts_pct_calc'
]
overall_averages = df_clustered.select([avg(c).alias(c) for c in feature_cols]).collect()[0].asDict()

cluster_profiles_df = get_cluster_profiles(df_clustered, feature_cols).toPandas().set_index('cluster_id')

def plot_cluster_comparison(cluster_id, stats_to_plot, cluster_profiles, overall_avg, color, save_name, title):
    data_list = []
    for stat in stats_to_plot:
        clean_stat_name = stat.replace('_per_36_min', '').replace('_calc', '').upper()
        data_list.append({'stat': clean_stat_name, 'type': f'Cluster {cluster_id}', 'value': cluster_profiles.loc[cluster_id, f'avg_{stat}']})
        data_list.append({'stat': clean_stat_name, 'type': 'Media Generale', 'value': overall_avg[stat]})
    
    plot_df = pd.DataFrame(data_list)
    
    plt.figure(figsize=(8, 5))
    sns.barplot(data=plot_df, x='stat', y='value', hue='type', palette=[color, 'lightgray'])
    plt.title(title, fontsize=14)
    plt.ylabel('Valore Medio Normalizzato')
    plt.xlabel('Statistica')
    plt.legend(title=None)
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_OUTPUT_PATH, save_name))
    plt.show()

plot_cluster_comparison(1, ['pts_per_36_min', 'trb_per_36_min'], cluster_profiles_df, overall_averages, '#1f77b4', 'slide_11_cluster_1.png', 'Cluster 1 (Marcatori-Rimbalzisti): Punti e Rimbalzi')
plot_cluster_comparison(2, ['ast_per_36_min', 'stl_per_36_min'], cluster_profiles_df, overall_averages, '#2ca02c', 'slide_12_cluster_2.png', 'Cluster 2 (Playmaker Puri): Assist e Palle Rubate')
plot_cluster_comparison(3, ['pts_per_36_min', 'ast_per_36_min'], cluster_profiles_df, overall_averages, '#d62728', 'slide_13_cluster_3.png', 'Cluster 3 (Giocatori di Ruolo): Contributo Generale')
plot_cluster_comparison(4, ['pts_per_36_min', 'ast_per_36_min'], cluster_profiles_df, overall_averages, '#ffd700', 'slide_14_cluster_4.png', 'Cluster 4 (All-Around Stars): Punti e Assist')
plot_cluster_comparison(5, ['tov_per_36_min'], cluster_profiles_df, overall_averages, '#ff7f0e', 'slide_15_cluster_5.png', 'Cluster 5 (Controllo Rischio): Palle Perse')
plot_cluster_comparison(6, ['trb_per_36_min', 'blk_per_36_min'], cluster_profiles_df, overall_averages, '#9467bd', 'slide_16_cluster_6.png', 'Cluster 6 (Ancore Difensive): Rimbalzi e Stoppate')

#### Interpretazione dei grafici

Questa serie di grafici mi permette di raccontare una storia chiara per ogni profilo:

- **Cluster 1 (Blu)**: Supera la media in punti e rimbalzi, confermando il suo ruolo di **Marcatore-Rimbalzista**.
- **Cluster 2 (Verde)**: Eccelle in assist e palle rubate, definendolo come **Playmaker Puro**.
- **Cluster 3 (Rosso)**: Le sue barre sono sotto la media, visualizzando il suo profilo di **Giocatore di Ruolo**.
- **Cluster 4 (Oro)**: Domina in punti e assist, evidenziando il suo ruolo di **All-Around Stars**.
- **Cluster 5 (Arancione)**: Si distingue per le bassissime palle perse, il suo marchio di fabbrica come **Giocatore a Controllo Rischio**.
- **Cluster 6 (Viola)**: Eccelle in rimbalzi e stoppate, definendolo come **Ancora Difensiva**.

Ogni grafico è perfetto per una slide della presentazione.

### Conclusione dei Notebook

Ho generato e salvato tutti i grafici che mi servono per comunicare efficacemente i risultati della mia analisi. Ogni visualizzazione è pronta per essere importata nelle slide. Il mio progetto, dalla preparazione dati alla comunicazione dei risultati, è ora completo.

In [ ]:
spark.stop()